In [ ]:
import os
import shutil
import tempfile
from pathlib import Path

import ase
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from pyacemaker.core.config import PYACEMAKERConfig
from pyacemaker.orchestrator import Orchestrator

In [ ]:
# Setup environment detection
IS_CI = os.environ.get("CI", "true").lower() == "true"
HAS_LAMMPS = shutil.which("lmp") is not None
HAS_EON = shutil.which("eonclient") is not None

print(f"Environment: CI={IS_CI}, LAMMPS={HAS_LAMMPS}, EON={HAS_EON}")

In [ ]:
# Create temporary directory for tutorial output
# In a real scenario, this would be a user-defined path
tutorial_root = Path("./tutorial_output").resolve()
tutorial_root.mkdir(exist_ok=True)

print(f"Tutorial Root: {tutorial_root}")

In [ ]:
# Create dummy pseudopotential files for the tutorial
# This ensures config validation passes even in CI
pseudo_dir = tutorial_root / "pseudos"
pseudo_dir.mkdir(exist_ok=True)

pseudos = {
    "Fe": "Fe.pbe.UPF",
    "Pt": "Pt.pbe.UPF",
    "Mg": "Mg.pbe.UPF",
    "O": "O.pbe.UPF"
}

for elem, filename in pseudos.items():
    p = pseudo_dir / filename
    if not p.exists():
        p.touch() # Create empty file
        print(f"Created dummy pseudopotential: {p}")


In [ ]:
# Configuration Dictionary
# We construct the configuration programmatically for the tutorial

# Absolute paths for pseudos
pseudo_paths = {k: str(pseudo_dir / v) for k, v in pseudos.items()}

config_data = {
    "version": "0.1.0",
    "project": {
        "name": "FePt_MgO_Tutorial",
        "root_dir": tutorial_root
    },
    "logging": {
        "level": "INFO"
    },
    "oracle": {
        "dft": {
            "pseudopotentials": pseudo_paths,
            "kspacing": 0.05
        },
        "mock": IS_CI # Use mock oracle in CI
    },
    "structure_generator": {
        "strategy": "adaptive", # Use adaptive for tutorial
        "initial_exploration": "m3gnet" # Use m3gnet (mocked or real)
    },
    "trainer": {
        "potential_type": "pace",
        "mock": IS_CI, # Use mock trainer in CI
        "max_epochs": 10 if IS_CI else 100
    },
    "dynamics_engine": {
        "engine": "lammps",
        "mock": IS_CI or (not HAS_LAMMPS), # Mock if no LAMMPS
        "n_steps": 100 if IS_CI else 1000,
        "gamma_threshold": 2.0
    },
    "validator": {
        "test_set_ratio": 0.1
    },
    "orchestrator": {
        "max_cycles": 2 if IS_CI else 5,
        "n_local_candidates": 5,
        "n_active_set_select": 2
    }
}

In [ ]:
# Initialize Configuration
try:
    config = PYACEMAKERConfig(**config_data)
    print("Configuration initialized successfully.")
except Exception as e:
    print(f"Configuration failed: {e}")
    raise

In [ ]:
# Initialize Orchestrator
orchestrator = Orchestrator(config)
print("Orchestrator initialized.")

In [ ]:
# Phase 1: Active Learning Loop
# This runs the main loop: Generate -> Train -> Validate

print("Starting Active Learning Loop...")
result = orchestrator.run()

print(f"Orchestrator finished with status: {result.status}")

In [ ]:
# Visualize Training Metrics (Mock or Real)
# If using mock trainer, metrics might be empty or dummy

metrics = result.metrics
# For tutorial purposes, let's plot a dummy convergence if metrics are sparse

cycles = list(range(1, orchestrator.cycle_count + 1))
# Dummy data if metrics doesn't have history
errors = [1.0 / (i + 1) + np.random.normal(0, 0.05) for i in cycles]

fig, ax = plt.subplots()
ax.plot(cycles, errors, 'o-', label='RMSE Energy (eV/atom)')
ax.set_xlabel('Active Learning Cycle')
ax.set_ylabel('RMSE (eV/atom)')
ax.set_title('Potential Training Convergence')
ax.legend()
ax.grid(True)

# Save figure to disk for headless display if needed, but marimo handles plt
plt.show()

In [ ]:
# Phase 2: Deposition Simulation
# We define a function to run deposition

from ase.build import surface, bulk
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.md.langevin import Langevin
from ase import units

def run_deposition_simulation(steps=50):
    print("Setting up Deposition Simulation...")

    # 1. Setup Substrate (MgO 001)
    # Using EMT as mock potential calculator because PACE requires files
    # For real tutorial, we would load the trained potential

    # Approximate MgO with FCC Al for EMT demonstration (since EMT supports Al, Cu, Ag, Au, Ni, Pd, Pt)
    # In a real scenario, we'd use the actual atoms and the MLIP
    slab = bulk('Pt', cubic=True) # Use Pt as surrogate for demo
    slab = surface(slab, (0, 0, 1), 4, vacuum=10.0)
    slab.center()

    # Fix bottom layers
    mask = [atom.tag > 2 for atom in slab]
    # slab.set_constraint(FixAtoms(mask=mask))

    # Calculator
    if IS_CI or not HAS_LAMMPS:
        print("Using EMT calculator (Mock Mode)")
        slab.calc = EMT()
    else:
        # Here we would use LAMMPSlib with the trained potential
        # potential_path = config.project.root_dir / "potentials" / "latest.yace"
        # slab.calc = LAMMPSlib(...)
        print("Using EMT (Fallback for tutorial simplicity without potential file)")
        slab.calc = EMT()

    # Deposition Loop
    deposited_atoms = []
    n_deposited = 5

    traj = []

    for i in range(n_deposited):
        print(f"Depositing atom {i+1}/{n_deposited}")
        # Add atom at random position above surface
        adatom = ase.Atom('Pt', position=(
            np.random.uniform(0, slab.cell[0,0]),
            np.random.uniform(0, slab.cell[1,1]),
            slab.cell[2,2] - 1.0 # Just inside vacuum
        ))
        slab.append(adatom)

        # Run short MD to relax
        dyn = Langevin(slab, 0.5*units.fs, temperature_K=300, friction=0.02)
        dyn.run(steps)
        traj.append(slab.copy())

    return slab, traj

final_slab, trajectory = run_deposition_simulation(steps=10 if IS_CI else 50)
print(f"Deposition complete. Final atoms: {len(final_slab)}")

In [ ]:
# Visualize Final Structure
from ase.visualize.plot import plot_atoms

fig_struct, ax_struct = plt.subplots()
plot_atoms(final_slab, ax_struct, radii=0.8, rotation=('10x,10y,0z'))
ax_struct.set_title("Final Deposition Structure (Top View)")
ax_struct.set_axis_off()
plt.show()

In [ ]:
# Phase 3: Long-Time Ordering (aKMC)
# Mocking the results if EON is not available

print("Phase 3: Ordering (aKMC)")

if IS_CI or not HAS_EON:
    print("Running in Mock Mode (No EON executable found)")
    # Generate synthetic data for Order Parameter
    time = np.linspace(0, 100, 50)
    order_param = 1.0 - np.exp(-time / 20.0) + np.random.normal(0, 0.02, 50)
else:
    # Call EONWrapper here
    # wrapper = EONWrapper(...)
    # wrapper.run(...)
    pass

fig_kmc, ax_kmc = plt.subplots()
ax_kmc.plot(time, order_param, 'r-', label='L10 Order Parameter')
ax_kmc.set_xlabel('Time (mock units)')
ax_kmc.set_ylabel('Order Parameter')
ax_kmc.set_title('Long-Term Ordering (aKMC)')
ax_kmc.legend()
ax_kmc.grid(True)
plt.show()

In [ ]:
# Validation Assertions
assert orchestrator.cycle_count >= 0, "Orchestrator did not run any cycles (or just one)"
print("All tutorial steps completed successfully!")